In [1]:
from layer import Dense
from optimizer import GradientDescent
from activation import ReLU, sigmoid
from metrics import MSE
from utils import batch_iterator

import prettytable
import numpy as np
import copy

In [2]:
class Module:

    def fit(self):
        raise NotImplementedError()

    def train(self, X, y, batch_size, epochs):
        raise NotImplementedError()

    def predict(self, X):
        raise NotImplementedError()


class NeuralNet:

    def __init__(self, optimizer, metrics):
        self.optimizer = copy.copy(optimizer)
        self.metrics = metrics()
        self.layers = []

    def add(self, layer):
        layer.set_optimizer(self.optimizer)
        if self.layers:
            layer.set_input_shape(self.layers[-1].output_shape())
        self.layers.append(layer)


    def fit(self):
        for layer in self.layers:
            layer.initialize()

    def train(self, X, y, batch_size=32, epochs=5000):
        loss = None
        y_pred = None

        for i in range(epochs):
            for batch_x, batch_y in batch_iterator(X, y, batch_size):
                y_pred = self._forward(batch_x)
                loss = self.metrics.loss(batch_y, y_pred)
                gradient = self.metrics.gradient(batch_y, y_pred)
                self._backward(gradient)

            if (i+1)%50 == 0:
#                 print(y_pred)
#                 print(batch_y)
#                 self.getweights()
                print(f" -> Loss : {loss} {'---------------'*5} at {i} / {epochs} ")
#             if i>30:
#                 break


    def fit_and_train(self, X, y, batch_size=32, epochs=5000):
        self.fit()
        self.train(X, y, batch_size, epochs)



    def _forward(self, X):
        layer_output = X

        for layer in self.layers:
            layer_output = layer.forward(layer_output)

        return layer_output


    def _backward(self, gradient):
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)


    def predict(self, X):
        return self._forward(X)
    
    def getweights(self):
        for layer in self.layers:
            layer.getweights()

    def summary(self):
        table = prettytable.PrettyTable()
        
        table.field_names = ["Layer No.", "Layer Name", "Layer Size", "Trainable Parameters", "Weights", "Bias"]

        for i, layer in enumerate(self.layers):
            layer_name, layer_size, trainable_params, weights, bias = layer.summary()
            table.add_row([i+1, layer_name, layer_size, trainable_params, weights, bias])

        print(table)
        
    def getparams(self):
        for layer in self.layers:
            layer.getparams()

In [5]:
x_train = np.random.uniform(0, 50, (5, 200))
y_train = 2*x_train[0] + 6*x_train[0]**2 + 2.5*x_train[3] + 0.1*x_train[4]
y_train = y_train.reshape(1, 200)

In [6]:
print(x_train)

[[3.85451254e+01 1.48291409e+01 1.56050948e+01 3.97134500e+01
  1.73470240e+01 3.07933896e+01 2.68535538e+00 9.41191619e+00
  4.23593694e+01 4.12343728e+01 4.55815165e+01 3.79563530e+01
  8.39581760e+00 3.02224747e+01 2.01672277e+00 2.90838991e+01
  4.33276296e+01 2.48307792e+01 3.76503346e+00 1.55382343e+01
  5.21092985e+00 4.26785695e+01 1.45863294e+01 1.07899801e+01
  8.02252552e+00 4.10717709e+01 9.69529144e+00 2.81384138e+01
  1.24017556e+01 3.47850199e+01 1.61759425e+01 2.52701210e+01
  4.87781525e+01 3.73145127e+01 2.21272553e+01 1.61842457e+01
  7.28944688e-01 3.72828128e+01 1.38911292e+01 3.51859956e+01
  1.86925490e+01 8.31685645e+00 3.41106858e+00 3.32954740e+01
  2.93000232e+01 2.70432109e+01 6.66137520e+00 2.65622454e+01
  3.33555650e+01 2.57775729e+01 3.53036085e+01 3.06893064e+01
  8.25098339e+00 5.93858080e+00 2.31418089e+00 2.14643684e+01
  4.68360033e+01 3.65929808e+01 9.58645752e-01 3.76937359e+01
  3.35390087e+01 3.37934073e+01 3.63336700e+01 5.86775406e+00
  3.4611

In [10]:
optimizer = GradientDescent(0.000092)
model = NeuralNet(optimizer, metrics=MSE)

model.add(Dense(32, (13,None)))
model.add(Dense(16))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(1))

9.2e-05
0
9.2e-05
0
9.2e-05
0
9.2e-05
0
9.2e-05
0


In [11]:
model.fit_and_train(x_train, y_train)

 -> Loss : [297.61825] --------------------------------------------------------------------------- at 49 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 99 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 149 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 199 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 249 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 299 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 349 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 399 / 5000 
 -> Loss : [297.61825] -------------------------------------------------------------------

 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3649 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3699 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3749 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3799 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3849 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3899 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3949 / 5000 
 -> Loss : [297.61825] --------------------------------------------------------------------------- at 3999 / 5000 
 -> Loss : [297.61825] ---------------------------------------------------------

In [ ]:
model.summary()

In [ ]:
model.getparams()

In [8]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path='boston_housing.npz', test_split=0.2, seed=113
)

In [ ]:
model = Sequential([
    Dense(8, input_dim=4, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='relu')
])

In [ ]:
# my model input = (n, m)
# tf model input = (m, n)
history = model.fit(X, y, batch_size=32, epochs=1000)

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
x_test = np.random.uniform(0, 50, (5, 4))
y_test = 2*x_test[0] + 6*x_test[0] + 2.5*x_test[3] + 0.1*x_test[4]
y_test = y_test.reshape(1, 4)

In [ ]:
model.predict(x_test)

In [ ]:
print(y_train.shape)

In [9]:
x_train = x_train.T
print(x_train.shape)

y_train = y_train.reshape(-1, 1)
y_train = y_train.T
print(y_train.shape)

(13, 404)
(1, 404)


In [ ]:
x_test = x_test.T
print(x_test.shape)

y_test = y_test.reshape(-1, 1)
y_test = y_test.T
print(y_test.shape)